<a href="https://colab.research.google.com/github/jre29/lecture1/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pobierz dane dotyczące scoringu kredytowego

*Pracujesz jako analityk danych w globalnej firmie finansowej. Przez lata firma gromadziła podstawowe dane bankowe i wiele informacji związanych z kredytami. Kierownictwo chce zbudować inteligentny system do segregowania ludzi w przedziałach oceny kredytowej, aby zmniejszyć wysiłek ręczny.*

**Zadanie na dziś: przygotować dane do modelu**

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import danych

In [63]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,
)

print("First 5 records:", df.head())

<ipython-input-63-db52f5663265>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,


First 5 records:        ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utiliz

/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


**Poznaj dane:**

1.   sprawdź wymiary i typy kolumn
2.   usuń duplikaty
3.   usuń zdbędne kolumny
4.   sprawdz liczbę braków danych w poszczególnych kolumnach






In [8]:
df.describe
df.dtypes
df.shape

(100000, 28)

In [64]:
df.drop_duplicates(inplace=True)

In [65]:
missing_values = df.isnull().sum()
print(missing_values)

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64


Usuń kolumny, które nie będą wykorzystywane w analizie ['ID','Customer_ID','SSN','Name']

In [66]:
df.drop(columns=['ID','Customer_ID','SSN','Name'],inplace=True)

In [67]:
df.shape

(100000, 24)

usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan

In [68]:
df = df.replace('', np.nan)

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str).str.replace('_', ' ').str.replace('-', ' ')

In [27]:
df.head(5)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,January,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit Builder Loan, Personal Loan,...",...,nan,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High spent Small value payments,312.49408867943663,Good
1,February,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit Builder Loan, Personal Loan,...",...,Good,809.98,31.944960,nan,No,49.574949,118.28022162236736,Low spent Large value payments,284.62916249607184,Good
2,March,500,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit Builder Loan, Personal Loan,...",...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low spent Medium value payments,331.2098628537912,Good
3,April,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit Builder Loan, Personal Loan,...",...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low spent Small value payments,223.45130972736786,Good
4,May,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit Builder Loan, Personal Loan,...",...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High spent Medium value payments,341.48923103222177,Good


In [28]:
df.dtypes

,0
Month,object
Age,object
Occupation,object
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64
Num_Credit_Card,int64
Interest_Rate,int64
Num_of_Loan,object
Type_of_Loan,object


Zmień nazwy miesiecy na zmienne liczbowe zgodnie z kolejnością miesięcy

In [69]:
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

df['Month'] = df['Month'].map(month_mapping)
df.fillna(0, inplace=True)
df['Month'] = df['Month'].astype(int)


In [52]:
df.dtypes

,0
Month,int64
Age,object
Occupation,object
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64
Num_Credit_Card,int64
Interest_Rate,int64
Num_of_Loan,object
Type_of_Loan,object


Zakoduj zmienne tekstowe: 'Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix', użyj LabelEncodera

In [70]:
to_encode = ['Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix']

for col in to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

Przelicz kolumne Credit_History_Age na liczbę mięsiecy


In [82]:
def convert_to_months(age_str):
    if pd.isnull(age_str):
        return np.nan

    years, months = 0, 0

    parts = age_str.split('and')
    for part in parts:
        if 'Years' in part:
            years = int(part.strip().split()[0])
        elif 'Months' in part:
            months = int(part.strip().split()[0])

    return years * 12 + months

df['Credit_History_Age'] = df['Credit_History_Age'].apply(convert_to_months)

In [83]:
print(df)

       Month   Age  Occupation Annual_Income  Monthly_Inhand_Salary  \
0          1    23          13      19114.12            1824.843333   
1          2    23          13      19114.12               0.000000   
2          3   500          13      19114.12               0.000000   
3          4    23          13      19114.12               0.000000   
4          5    23          13      19114.12            1824.843333   
...      ...   ...         ...           ...                    ...   
99995      4    25          10      39628.99            3359.415833   
99996      5    25          10      39628.99            3359.415833   
99997      6    25          10      39628.99            3359.415833   
99998      7    25          10      39628.99            3359.415833   
99999      8    25          10     39628.99             3359.415833   

       Num_Bank_Accounts  Num_Credit_Card  Interest_Rate Num_of_Loan  \
0                      3                4              3           4   
1  

In [58]:
df['Type_of_Loan'] = df['Type_of_Loan'].fillna('')
df['Type_of_Loan'] = df['Type_of_Loan'].str.replace(r'\band\b', '', regex=True).str.strip() #removes "and"

# Split and clean the values
df_type_of_loan = df['Type_of_Loan'].str.split(r',\s*')  # Handles different spacing

# Use MultiLabelBinarizer
mlb = MultiLabelBinarizer()
dt = mlb.fit_transform(df_type_of_loan.to_list())  # Ensure correct format

# Convert to DataFrame
encoded_df = pd.DataFrame(data=dt, columns=mlb.classes_)

# Merge with original DataFrame and drop old column
df = df.join(encoded_df).drop(columns=['Type_of_Loan'])

print(df.head())

AttributeError: Can only use .str accessor with string values!

In [84]:
df.dtypes

,0
Month,int64
Age,object
Occupation,int64
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64
Num_Credit_Card,int64
Interest_Rate,int64
Num_of_Loan,object
Type_of_Loan,int64


Zmien inne zmienne na numeryczne

In [85]:
numeric_cols = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
                'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
                'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
                'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio',
                'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']

for col in numeric_cols:
    if col in df.columns:
        # Próba konwersji na liczby
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [88]:
df.dtypes

,0
Month,int64
Age,int64
Occupation,int64
Annual_Income,float64
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64
Num_Credit_Card,int64
Interest_Rate,int64
Num_of_Loan,int64
Type_of_Loan,int64


Uzupełnij braki w kolumnie Monthly_Inhand_Salary,  Annual_Income, .. średnią

Narysuj rozkłady zmiennych 'Annual_Income', 'Monthly_Balance'

usuń wartości odstające (w przypadku wieku wszystkie ujemne i te powyzej 100 lat)

Zwizualizuj korelacje między zmiennymi